In [4]:
!pip install selenium pandas tqdm


  Using cached websocket_client-1.8.0-py3-none-any.whl.metadata (8.0 kB)
  Using cached cffi-1.17.1-cp310-cp310-win_amd64.whl.metadata (1.6 kB)
  Using cached pycparser-2.22-py3-none-any.whl.metadata (943 bytes)
   ---------------------------------------- 0.0/9.4 MB ? eta -:--:--
   ----------------------- ---------------- 5.5/9.4 MB 25.8 MB/s eta 0:00:01
   ---------------------------------------- 9.4/9.4 MB 29.3 MB/s eta 0:00:00
Using cached websocket_client-1.8.0-py3-none-any.whl (58 kB)
Using cached cffi-1.17.1-cp310-cp310-win_amd64.whl (181 kB)
Using cached pycparser-2.22-py3-none-any.whl (117 kB)

   -------- -------------------------------  2/10 [websocket-client]
   ------------ ---------------------------  3/10 [pysocks]
   ---------------- -----------------------  4/10 [pycparser]
   ---------------- -----------------------  4/10 [pycparser]
   ------------------------ ---------------  6/10 [cffi]
   ---------------------------- -----------  7/10 [trio]
   -------------------

In [8]:
import FinanceDataReader as fdr
import pandas as pd
# 상장폐지 종목 목록 가져오기
delisted = fdr.StockListing('KRX-DELISTING')
# 데이터프레임 열 이름 확인
print("상장폐지 데이터 열 이름:")
print(delisted.columns)
# 종목코드 열 이름 설정
code_column = 'Code' if 'Code' in delisted.columns else 'Symbol'
# 종목코드 필터링: 6자리 숫자이고 마지막 자리가 0
delisted = delisted[
    # 6자리 숫자 확인
    delisted[code_column].str.match(r'^\d{6}$') &
    # 마지막 자리가 0
    delisted[code_column].str.endswith('0') &
    # 2005년 이후 상장폐지
    (delisted['DelistingDate'] >= '2005-01-01')
]
# 기업 자체 상장폐지 사유 필터링
# 제외할 사유: 합병, 스팩 합병, 자진상장폐지
# 제외할 종목: 우선주, 스팩
# include_reasons = ['경영악화', '재무구조악화', '감사의견', '사업보고서']
exclude_reasons = ['피흡수합병', '완전자회사화', '완전자회사로 편입', '존속기간만료', '존속기간 만료', '해산 사유 발생']
exclude_preferred = ['우선주', '스팩']
# 필터링 조건
filtered_delisted = delisted[
    # 합병, 자회사화, 스팩 제외
    ~delisted['Reason'].str.contains('|'.join(exclude_reasons), case=False, na=False) &
    # 우선주, 스팩 제외
    ~delisted['Name'].str.contains('|'.join(exclude_preferred), case=False, na=False)
]
# 상장폐지 날짜로 정렬 (최신순)
filtered_delisted['DelistingDate'] = pd.to_datetime(filtered_delisted['DelistingDate'], errors='coerce')
sorted_delisted = filtered_delisted.sort_values(by='DelistingDate', ascending=False)
# 필요한 열만 선택
columns_to_show = [code_column, 'Name', 'Market', 'DelistingDate', 'Reason']
if all(col in filtered_delisted.columns for col in columns_to_show):
    sorted_delisted = sorted_delisted[columns_to_show]
else:
    print("일부 열이 누락되었습니다. 사용 가능한 열:", filtered_delisted.columns)
# 상위 10개 출력
print("\n최신 상장폐지 종목 목록 (상위 10개):")
print(sorted_delisted.head(10))
# CSV 파일로 저장
sorted_delisted.to_csv('src_data/delisted_stocks.csv', index=False, encoding='utf-8-sig')
print("\n최신순으로 정렬된 상장폐지 종목 목록이 'delisted_stocks.csv' 파일로 저장되었습니다.")
print(len(filtered_delisted))

ModuleNotFoundError: No module named 'FinanceDataReader'

## 수정된 코드

In [10]:
!pip install FinanceDataReader


ERROR: Could not find a version that satisfies the requirement FinanceDataReader (from versions: none)
ERROR: No matching distribution found for FinanceDataReader


In [11]:
! pip install FinanceDataReader


ERROR: Could not find a version that satisfies the requirement FinanceDataReader (from versions: none)
ERROR: No matching distribution found for FinanceDataReader


In [14]:
!pip install git+https://github.com/FinanceData/FinanceDataReader.git


  Cloning https://github.com/FinanceData/FinanceDataReader.git to c:\users\jeons\appdata\local\temp\pip-req-build-i0b2g31a
  Resolved https://github.com/FinanceData/FinanceDataReader.git to commit 79b77abf4633dd60f3d4ff1519f8fae1640509b8
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached beautifulsoup4-4.13.4-py3-none-any.whl.metadata (3.8 kB)
  Using cached soupsieve-2.7-py3-none-any.whl.metadata (4.6 kB)
Using cached beautifulsoup4-4.13.4-py3-none-any.whl (187 kB)
Using cached soupsieve-2.7-py3-none-any.whl (36 kB)
   ---------------------------------------- 0.0/4.0 MB ? eta -:--:--
   ---------------------------------------- 4.0/4.0 MB 33.9 MB/s eta 0:00:00
   ----------------------------------

  Running command git clone --filter=blob:none --quiet https://github.com/FinanceData/FinanceDataReader.git 'C:\Users\jeons\AppData\Local\Temp\pip-req-build-i0b2g31a'


## 2005-2015

In [15]:
import FinanceDataReader as fdr
import pandas as pd
import os

# 📁 저장 경로 설정
save_path = 'src_data'
os.makedirs(save_path, exist_ok=True)

# ✅ 1. 상장폐지 종목 목록 가져오기
delisted = fdr.StockListing('KRX-DELISTING')
print("✅ 전체 상장폐지 종목 수:", len(delisted))

# ✅ 2. 열 이름 확인 및 종목코드 열 설정
print("\n📌 열 이름 목록:")
print(delisted.columns)
code_column = 'Code' if 'Code' in delisted.columns else 'Symbol'

# ✅ 3. 필터링: 6자리 종목코드 + 마지막 숫자 0 + 2005년 이후 폐지
delisted = delisted[
    delisted[code_column].str.match(r'^\d{6}$') &
    delisted[code_column].str.endswith('0') &
    (delisted['DelistingDate'] >= '2005-01-01')
]

# ✅ 4. 제외할 상장폐지 사유 및 종목명
exclude_reasons = ['피흡수합병', '완전자회사화', '완전자회사로 편입', '존속기간만료', '존속기간 만료', '해산 사유 발생']
exclude_keywords = ['우선주', '스팩']

# ✅ 5. 사유와 종목명 기준 필터링
filtered_delisted = delisted[
    ~delisted['Reason'].str.contains('|'.join(exclude_reasons), case=False, na=False) &
    ~delisted['Name'].str.contains('|'.join(exclude_keywords), case=False, na=False)
]

# ✅ 6. 날짜 형식 변환 및 정렬
filtered_delisted['DelistingDate'] = pd.to_datetime(filtered_delisted['DelistingDate'], errors='coerce')
sorted_delisted = filtered_delisted.sort_values(by='DelistingDate', ascending=False)

# ✅ 7. 최종 열 구성
columns_to_show = [code_column, 'Name', 'Market', 'DelistingDate', 'Reason']
if all(col in sorted_delisted.columns for col in columns_to_show):
    sorted_delisted = sorted_delisted[columns_to_show]
else:
    print("⚠ 일부 열이 누락되었습니다. 사용 가능한 열:")
    print(sorted_delisted.columns)

# ✅ 8. 출력 & 저장
print("\n📋 최신 상장폐지 종목 (상위 10개):")
print(sorted_delisted.head(10))

file_path = os.path.join(save_path, 'delisted_stocks.csv')
sorted_delisted.to_csv(file_path, index=False, encoding='utf-8-sig')
print(f"\n✅ CSV 저장 완료 → '{file_path}'")
print(f"✅ 최종 필터링된 상장폐지 기업 수: {len(filtered_delisted)}")


✅ 전체 상장폐지 종목 수: 3916

📌 열 이름 목록:
Index(['Symbol', 'Name', 'Market', 'SecuGroup', 'Kind', 'ListingDate',
       'DelistingDate', 'Reason', 'ArrantEnforceDate', 'ArrantEndDate',
       'Industry', 'ParValue', 'ListingShares', 'ToSymbol', 'ToName'],
      dtype='object')

📋 최신 상장폐지 종목 (상위 10개):
      Symbol      Name  Market DelistingDate  \
3746  023460       CNH  KOSDAQ    2025-07-14   
3757  226440     한송네오텍  KOSDAQ    2025-06-19   
3756  323230   엠에프엠코리아  KOSDAQ    2025-06-19   
3758  071460       위니아  KOSDAQ    2025-06-18   
3759  072520     제넨바이오  KOSDAQ    2025-06-18   
3765  283100      노보믹스   KONEX    2025-05-28   
3766  012600     청호ICT   KOSPI    2025-05-27   
3776  258540  에스엘테라퓨틱스   KONEX    2025-05-07   
3795  214870     한울BnC  KOSDAQ    2025-03-31   
3797  038340       MIT  KOSDAQ    2025-03-28   

                                                 Reason  
3746   기업의 계속성 및 경영의 투명성 등을 종합적으로 고려하여 상장폐지기준에 해당한다고 결정  
3757   기업의 계속성 및 경영의 투명성 등을 종합적으로 고려하여 상장폐지기준에 해당한다고 결정  
3756

C:\Users\jeons\AppData\Local\Temp\ipykernel_20264\4266590220.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_delisted['DelistingDate'] = pd.to_datetime(filtered_delisted['DelistingDate'], errors='coerce')


## 2015-2025

In [ ]:
import FinanceDataReader as fdr
import pandas as pd
import os

# 📁 저장 경로 설정
save_path = 'src_data'
os.makedirs(save_path, exist_ok=True)

# ✅ 1. 상장폐지 종목 목록 가져오기
delisted = fdr.StockListing('KRX-DELISTING')
print("✅ 전체 상장폐지 종목 수:", len(delisted))

# ✅ 2. 열 이름 확인 및 종목코드 열 설정
print("\n📌 열 이름 목록:")
print(delisted.columns)
code_column = 'Code' if 'Code' in delisted.columns else 'Symbol'

# ✅ 3. 필터링: 6자리 종목코드 + 마지막 숫자 0 + 2005년 이후 폐지
delisted = delisted[
    delisted[code_column].str.match(r'^\d{6}$') &
    delisted[code_column].str.endswith('0') &
    (delisted['DelistingDate'] >= '2015-01-01')
]

# ✅ 4. 제외할 상장폐지 사유 및 종목명
exclude_reasons = ['피흡수합병', '완전자회사화', '완전자회사로 편입', '존속기간만료', '존속기간 만료', '해산 사유 발생']
exclude_keywords = ['우선주', '스팩']

# ✅ 5. 사유와 종목명 기준 필터링
filtered_delisted = delisted[
    ~delisted['Reason'].str.contains('|'.join(exclude_reasons), case=False, na=False) &
    ~delisted['Name'].str.contains('|'.join(exclude_keywords), case=False, na=False)
]

# ✅ 6. 날짜 형식 변환 및 정렬
filtered_delisted['DelistingDate'] = pd.to_datetime(filtered_delisted['DelistingDate'], errors='coerce')
sorted_delisted = filtered_delisted.sort_values(by='DelistingDate', ascending=False)

# ✅ 7. 최종 열 구성
columns_to_show = [code_column, 'Name', 'Market', 'DelistingDate', 'Reason']
if all(col in sorted_delisted.columns for col in columns_to_show):
    sorted_delisted = sorted_delisted[columns_to_show]
else:
    print("⚠ 일부 열이 누락되었습니다. 사용 가능한 열:")

    print(sorted_delisted.columns)

# ✅ 8. 출력 & 저장
print("\n📋 최신 상장폐지 종목 (상위 10개):")
print(sorted_delisted.head(10))

file_path = os.path.join(save_path, 'delisted_2015_2025.csv')
sorted_delisted.to_csv(file_path, index=False, encoding='utf-8-sig')
print(f"\n✅ CSV 저장 완료 → '{file_path}'")
print(f"✅ 최종 필터링된 상장폐지 기업 수: {len(filtered_delisted)}")


✅ 전체 상장폐지 종목 수: 3916

📌 열 이름 목록:
Index(['Symbol', 'Name', 'Market', 'SecuGroup', 'Kind', 'ListingDate',
       'DelistingDate', 'Reason', 'ArrantEnforceDate', 'ArrantEndDate',
       'Industry', 'ParValue', 'ListingShares', 'ToSymbol', 'ToName'],
      dtype='object')

📋 최신 상장폐지 종목 (상위 10개):
      Symbol      Name  Market DelistingDate  \
3746  023460       CNH  KOSDAQ    2025-07-14   
3756  323230   엠에프엠코리아  KOSDAQ    2025-06-19   
3757  226440     한송네오텍  KOSDAQ    2025-06-19   
3758  071460       위니아  KOSDAQ    2025-06-18   
3759  072520     제넨바이오  KOSDAQ    2025-06-18   
3765  283100      노보믹스   KONEX    2025-05-28   
3766  012600     청호ICT   KOSPI    2025-05-27   
3776  258540  에스엘테라퓨틱스   KONEX    2025-05-07   
3795  214870     한울BnC  KOSDAQ    2025-03-31   
3797  038340       MIT  KOSDAQ    2025-03-28   

                                                 Reason  
3746   기업의 계속성 및 경영의 투명성 등을 종합적으로 고려하여 상장폐지기준에 해당한다고 결정  
3756   기업의 계속성 및 경영의 투명성 등을 종합적으로 고려하여 상장폐지기준에 해당한다고 결정  
3757

C:\Users\jeons\AppData\Local\Temp\ipykernel_20264\36335517.py:36: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_delisted['DelistingDate'] = pd.to_datetime(filtered_delisted['DelistingDate'], errors='coerce')


In [19]:
import FinanceDataReader as fdr
import pandas as pd
import os
import datetime

# 📁 저장 경로 설정
save_path = 'src_data'
os.makedirs(save_path, exist_ok=True)

# ✅ 1. 상장폐지 종목 전체 목록 불러오기
delisted = fdr.StockListing('KRX-DELISTING')
print("✅ 전체 상장폐지 종목 수:", len(delisted))

# ✅ 2. 종목코드 열 설정 (Code 또는 Symbol 중 선택)
code_column = 'Code' if 'Code' in delisted.columns else 'Symbol'

# ✅ 3. 최근 10년 기준 날짜 계산
ten_years_ago = datetime.datetime.now() - datetime.timedelta(days=365 * 10)

# ✅ 4. 기본 필터링: 6자리 종목코드, 마지막 자리 0, 최근 10년 이내 폐지
delisted = delisted[
    delisted[code_column].str.match(r'^\d{6}$') &
    delisted[code_column].str.endswith('0') &
    (pd.to_datetime(delisted['DelistingDate'], errors='coerce') >= ten_years_ago)
]

# ✅ 5. 제외할 상장폐지 사유 및 종목명
exclude_reasons = ['피흡수합병', '완전자회사화', '완전자회사로 편입', '존속기간만료', '존속기간 만료', '해산 사유 발생']
exclude_keywords = ['우선주', '스팩']

filtered_delisted = delisted[
    ~delisted['Reason'].str.contains('|'.join(exclude_reasons), case=False, na=False) &
    ~delisted['Name'].str.contains('|'.join(exclude_keywords), case=False, na=False)
]

# ✅ 6. 날짜 변환 및 정렬
filtered_delisted['DelistingDate'] = pd.to_datetime(filtered_delisted['DelistingDate'], errors='coerce')
sorted_delisted = filtered_delisted.sort_values(by='DelistingDate', ascending=False)

# ✅ 7. 폐지 연도, 월, 일 분리 + 한글 표기용 열 추가
sorted_delisted['폐지연도'] = sorted_delisted['DelistingDate'].dt.year
sorted_delisted['폐지월'] = sorted_delisted['DelistingDate'].dt.month
sorted_delisted['폐지일'] = sorted_delisted['DelistingDate'].dt.day
sorted_delisted['폐지일자(표기용)'] = sorted_delisted['DelistingDate'].dt.strftime('%Y년 %m월 %d일')

# ✅ 8. 최종 열 구성
columns_to_show = [
    code_column, 'Name', 'Market', 'DelistingDate',
    '폐지연도', '폐지월', '폐지일', '폐지일자(표기용)', 'Reason'
]
sorted_delisted = sorted_delisted[columns_to_show]

# ✅ 9. 출력 및 CSV 저장
print("\n📋 최근 10년간 상장폐지 종목 (상위 10개):")
print(sorted_delisted.head(10))

file_path = os.path.join(save_path, 'delisted_stocks_10yrs.csv')
sorted_delisted.to_csv(file_path, index=False, encoding='utf-8-sig')
print(f"\n✅ CSV 저장 완료 → '{file_path}'")
print(f"✅ 최종 필터링된 기업 수: {len(sorted_delisted)}")


✅ 전체 상장폐지 종목 수: 3916

📋 최근 10년간 상장폐지 종목 (상위 10개):
      Symbol      Name  Market DelistingDate  폐지연도  폐지월  폐지일      폐지일자(표기용)  \
3746  023460       CNH  KOSDAQ    2025-07-14  2025    7   14  2025년 07월 14일   
3757  226440     한송네오텍  KOSDAQ    2025-06-19  2025    6   19  2025년 06월 19일   
3756  323230   엠에프엠코리아  KOSDAQ    2025-06-19  2025    6   19  2025년 06월 19일   
3758  071460       위니아  KOSDAQ    2025-06-18  2025    6   18  2025년 06월 18일   
3759  072520     제넨바이오  KOSDAQ    2025-06-18  2025    6   18  2025년 06월 18일   
3765  283100      노보믹스   KONEX    2025-05-28  2025    5   28  2025년 05월 28일   
3766  012600     청호ICT   KOSPI    2025-05-27  2025    5   27  2025년 05월 27일   
3776  258540  에스엘테라퓨틱스   KONEX    2025-05-07  2025    5    7  2025년 05월 07일   
3795  214870     한울BnC  KOSDAQ    2025-03-31  2025    3   31  2025년 03월 31일   
3797  038340       MIT  KOSDAQ    2025-03-28  2025    3   28  2025년 03월 28일   

                                                 Reason  
3746   기업의 계속성 및 경영의 투

C:\Users\jeons\AppData\Local\Temp\ipykernel_20264\2923713237.py:37: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_delisted['DelistingDate'] = pd.to_datetime(filtered_delisted['DelistingDate'], errors='coerce')


## 폐지날짜 개선 코드

In [20]:
import FinanceDataReader as fdr
import pandas as pd
import os
import datetime

# 저장 폴더 생성
save_path = 'src_data'
os.makedirs(save_path, exist_ok=True)

# 1. 전체 상장폐지 종목 불러오기
delisted = fdr.StockListing('KRX-DELISTING')

# 2. 종목코드 열 설정
code_column = 'Code' if 'Code' in delisted.columns else 'Symbol'

# 3. 날짜 형식 변환 (필수!)
delisted['DelistingDate'] = pd.to_datetime(delisted['DelistingDate'], errors='coerce')

# 4. 최근 10년 기준 계산
ten_years_ago = datetime.datetime.now() - datetime.timedelta(days=365 * 10)

# 5. 필터링: 6자리 종목코드, 마지막 자리 0, 최근 10년
delisted = delisted[
    delisted[code_column].str.match(r'^\d{6}$') &
    delisted[code_column].str.endswith('0') &
    (delisted['DelistingDate'] >= ten_years_ago)
]

# 6. 제외 조건 적용
exclude_reasons = ['피흡수합병', '완전자회사화', '완전자회사로 편입', '존속기간만료', '존속기간 만료', '해산 사유 발생']
exclude_keywords = ['우선주', '스팩']

filtered_delisted = delisted[
    ~delisted['Reason'].str.contains('|'.join(exclude_reasons), case=False, na=False) &
    ~delisted['Name'].str.contains('|'.join(exclude_keywords), case=False, na=False)
]

# 7. 폐지일자 정보 분해
filtered_delisted['폐지연도'] = filtered_delisted['DelistingDate'].dt.year
filtered_delisted['폐지월'] = filtered_delisted['DelistingDate'].dt.month
filtered_delisted['폐지일'] = filtered_delisted['DelistingDate'].dt.day
filtered_delisted['폐지일자(표기용)'] = filtered_delisted['DelistingDate'].dt.strftime('%Y년 %m월 %d일')

# 8. 정렬 및 열 선택
sorted_delisted = filtered_delisted.sort_values(by='DelistingDate', ascending=False)
columns_to_show = [
    code_column, 'Name', 'Market', 'DelistingDate',
    '폐지연도', '폐지월', '폐지일', '폐지일자(표기용)', 'Reason'
]
sorted_delisted = sorted_delisted[columns_to_show]

# 9. 출력 및 저장
print("\n📋 최근 10년간 상장폐지 종목 (상위 10개):")
print(sorted_delisted.head(10))

file_path = os.path.join(save_path, 'delisted_stocks_10yrs.csv')
sorted_delisted.to_csv(file_path, index=False, encoding='utf-8-sig')
print(f"\n✅ CSV 저장 완료 → '{file_path}'")
print(f"✅ 최종 필터링된 기업 수: {len(sorted_delisted)}")



📋 최근 10년간 상장폐지 종목 (상위 10개):
      Symbol      Name  Market DelistingDate  폐지연도  폐지월  폐지일      폐지일자(표기용)  \
3746  023460       CNH  KOSDAQ    2025-07-14  2025    7   14  2025년 07월 14일   
3757  226440     한송네오텍  KOSDAQ    2025-06-19  2025    6   19  2025년 06월 19일   
3756  323230   엠에프엠코리아  KOSDAQ    2025-06-19  2025    6   19  2025년 06월 19일   
3758  071460       위니아  KOSDAQ    2025-06-18  2025    6   18  2025년 06월 18일   
3759  072520     제넨바이오  KOSDAQ    2025-06-18  2025    6   18  2025년 06월 18일   
3765  283100      노보믹스   KONEX    2025-05-28  2025    5   28  2025년 05월 28일   
3766  012600     청호ICT   KOSPI    2025-05-27  2025    5   27  2025년 05월 27일   
3776  258540  에스엘테라퓨틱스   KONEX    2025-05-07  2025    5    7  2025년 05월 07일   
3795  214870     한울BnC  KOSDAQ    2025-03-31  2025    3   31  2025년 03월 31일   
3797  038340       MIT  KOSDAQ    2025-03-28  2025    3   28  2025년 03월 28일   

                                                 Reason  
3746   기업의 계속성 및 경영의 투명성 등을 종합적으로 고려하여 상장폐지

C:\Users\jeons\AppData\Local\Temp\ipykernel_20264\1804988993.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_delisted['폐지연도'] = filtered_delisted['DelistingDate'].dt.year
C:\Users\jeons\AppData\Local\Temp\ipykernel_20264\1804988993.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_delisted['폐지월'] = filtered_delisted['DelistingDate'].dt.month
C:\Users\jeons\AppData\Local\Temp\ipykernel_20264\1804988993.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 

In [21]:
import FinanceDataReader as fdr
import pandas as pd
import os
import datetime

# 저장 폴더 생성
save_path = 'src_data'
os.makedirs(save_path, exist_ok=True)

# 1. 전체 상장폐지 종목 불러오기
delisted = fdr.StockListing('KRX-DELISTING')

# 2. 종목코드 열 설정
code_column = 'Code' if 'Code' in delisted.columns else 'Symbol'

# 3. 날짜 형식 변환 (필수!)
delisted['DelistingDate'] = pd.to_datetime(delisted['DelistingDate'], errors='coerce')

# ✅ 4. 2005~2010년 폐지 종목 필터링
start_date = datetime.datetime(2005, 1, 1)
end_date = datetime.datetime(2010, 12, 31)

delisted = delisted[
    delisted[code_column].str.match(r'^\d{6}$') &
    delisted[code_column].str.endswith('0') &
    (delisted['DelistingDate'] >= start_date) &
    (delisted['DelistingDate'] <= end_date)
]

# 5. 제외 조건 적용
exclude_reasons = ['피흡수합병', '완전자회사화', '완전자회사로 편입', '존속기간만료', '존속기간 만료', '해산 사유 발생']
exclude_keywords = ['우선주', '스팩']

filtered_delisted = delisted[
    ~delisted['Reason'].str.contains('|'.join(exclude_reasons), case=False, na=False) &
    ~delisted['Name'].str.contains('|'.join(exclude_keywords), case=False, na=False)
]

# 6. 폐지일자 정보 분해
filtered_delisted['폐지연도'] = filtered_delisted['DelistingDate'].dt.year
filtered_delisted['폐지월'] = filtered_delisted['DelistingDate'].dt.month
filtered_delisted['폐지일'] = filtered_delisted['DelistingDate'].dt.day
filtered_delisted['폐지일자(표기용)'] = filtered_delisted['DelistingDate'].dt.strftime('%Y년 %m월 %d일')

# 7. 정렬 및 열 선택
sorted_delisted = filtered_delisted.sort_values(by='DelistingDate', ascending=False)
columns_to_show = [
    code_column, 'Name', 'Market', 'DelistingDate',
    '폐지연도', '폐지월', '폐지일', '폐지일자(표기용)', 'Reason'
]
sorted_delisted = sorted_delisted[columns_to_show]

# 8. 출력 및 저장
print("\n📋 2005~2010년 상장폐지 종목 (상위 10개):")
print(sorted_delisted.head(10))

file_path = os.path.join(save_path, 'delisted_stocks_2005_2010.csv')
sorted_delisted.to_csv(file_path, index=False, encoding='utf-8-sig')
print(f"\n✅ CSV 저장 완료 → '{file_path}'")
print(f"✅ 최종 필터링된 기업 수: {len(sorted_delisted)}")



📋 2005~2010년 상장폐지 종목 (상위 10개):
      Symbol     Name  Market DelistingDate  폐지연도  폐지월  폐지일      폐지일자(표기용)  \
2057  052210   엠씨티티코어  KOSDAQ    2010-12-04  2010   12    4  2010년 12월 04일   
2058  101380    거북선2호   KOSPI    2010-12-03  2010   12    3  2010년 12월 03일   
2059  102000    거북선3호   KOSPI    2010-12-03  2010   12    3  2010년 12월 03일   
2060  047450      이앤텍  KOSDAQ    2010-11-30  2010   11   30  2010년 11월 30일   
2062  078420    동북아1호   KOSPI    2010-11-29  2010   11   29  2010년 11월 29일   
2063  048140    태광이엔시  KOSDAQ    2010-11-25  2010   11   25  2010년 11월 25일   
2065  048510      테스텍  KOSDAQ    2010-11-10  2010   11   10  2010년 11월 10일   
2066  070080  하이드로젠파워  KOSDAQ    2010-11-06  2010   11    6  2010년 11월 06일   
2068  080410    동북아6호   KOSPI    2010-10-25  2010   10   25  2010년 10월 25일   
2070  052970    올리브나인  KOSDAQ    2010-10-20  2010   10   20  2010년 10월 20일   

                                                 Reason  
2057  기업의 계속성, 경영의 투명성 또는 기타 코스닥시장의 건전성 등을 종합적으로 고려

C:\Users\jeons\AppData\Local\Temp\ipykernel_20264\491441661.py:40: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_delisted['폐지연도'] = filtered_delisted['DelistingDate'].dt.year
C:\Users\jeons\AppData\Local\Temp\ipykernel_20264\491441661.py:41: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_delisted['폐지월'] = filtered_delisted['DelistingDate'].dt.month
C:\Users\jeons\AppData\Local\Temp\ipykernel_20264\491441661.py:42: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice fro

## 2005-2025년 통합

In [28]:
import FinanceDataReader as fdr
import pandas as pd
import os
import datetime

# 저장 폴더 생성
save_path = 'src_data'
os.makedirs(save_path, exist_ok=True)

# 1. 전체 상장폐지 종목 불러오기
delisted = fdr.StockListing('KRX-DELISTING')

# 🔍 열 이름 확인
print("\n📌 delisted.columns:")
print(delisted.columns)

# ✅ 종목코드 열 정확히 확인 후 지정
code_column = 'Symbol'  # 예: 'Code', 'Symbol'

# ✅ 종목코드 6자리 문자열로 통일
delisted[code_column] = delisted[code_column].astype(str).str.zfill(6)

# 3. 날짜 형식 변환
delisted['DelistingDate'] = pd.to_datetime(delisted['DelistingDate'], errors='coerce')

# 4. 2005~2025년 폐지 종목 필터링
start_date = datetime.datetime(2005, 1, 1)
end_date = datetime.datetime(2025, 12, 31)

delisted = delisted[
    delisted[code_column].str.match(r'^\d{6}$') &
    delisted[code_column].str.endswith('0') &
    (delisted['DelistingDate'] >= start_date) &
    (delisted['DelistingDate'] <= end_date)
]

# 5. 제외 조건 적용
exclude_reasons = ['피흡수합병', '완전자회사화', '완전자회사로 편입', '존속기간만료', '존속기간 만료', '해산 사유 발생']
exclude_keywords = ['우선주', '스팩']

filtered_delisted = delisted[
    ~delisted['Reason'].str.contains('|'.join(exclude_reasons), case=False, na=False) &
    ~delisted['Name'].str.contains('|'.join(exclude_keywords), case=False, na=False)
]

# 6. 폐지일자 정보 분해
filtered_delisted['폐지연도'] = filtered_delisted['DelistingDate'].dt.year
filtered_delisted['폐지월'] = filtered_delisted['DelistingDate'].dt.month
filtered_delisted['폐지일'] = filtered_delisted['DelistingDate'].dt.day
filtered_delisted['폐지일자(표기용)'] = filtered_delisted['DelistingDate'].dt.strftime('%Y년 %m월 %d일')

# 7. 정렬 및 열 선택
sorted_delisted = filtered_delisted.sort_values(by='DelistingDate', ascending=False)
columns_to_show = [
    code_column, 'Name', 'Market', 'DelistingDate',
    '폐지연도', '폐지월', '폐지일', '폐지일자(표기용)', 'Reason'
]
sorted_delisted = sorted_delisted[columns_to_show]

# 8. 출력 및 저장
print("\n📋 2005~2025년 상장폐지 종목 (상위 10개):")
print(sorted_delisted.head(10))

file_path = os.path.join(save_path, 'delisted_stocks_2005_2025.csv')
sorted_delisted.to_csv(file_path, index=False, encoding='utf-8-sig')
print(f"\n✅ CSV 저장 완료 → '{file_path}'")
print(f"✅ 최종 필터링된 기업 수: {len(sorted_delisted)}")



📌 delisted.columns:
Index(['Symbol', 'Name', 'Market', 'SecuGroup', 'Kind', 'ListingDate',
       'DelistingDate', 'Reason', 'ArrantEnforceDate', 'ArrantEndDate',
       'Industry', 'ParValue', 'ListingShares', 'ToSymbol', 'ToName'],
      dtype='object')

📋 2005~2025년 상장폐지 종목 (상위 10개):
      Symbol      Name  Market DelistingDate  폐지연도  폐지월  폐지일      폐지일자(표기용)  \
3746  023460       CNH  KOSDAQ    2025-07-14  2025    7   14  2025년 07월 14일   
3757  226440     한송네오텍  KOSDAQ    2025-06-19  2025    6   19  2025년 06월 19일   
3756  323230   엠에프엠코리아  KOSDAQ    2025-06-19  2025    6   19  2025년 06월 19일   
3758  071460       위니아  KOSDAQ    2025-06-18  2025    6   18  2025년 06월 18일   
3759  072520     제넨바이오  KOSDAQ    2025-06-18  2025    6   18  2025년 06월 18일   
3765  283100      노보믹스   KONEX    2025-05-28  2025    5   28  2025년 05월 28일   
3766  012600     청호ICT   KOSPI    2025-05-27  2025    5   27  2025년 05월 27일   
3776  258540  에스엘테라퓨틱스   KONEX    2025-05-07  2025    5    7  2025년 05월 07일   


C:\Users\jeons\AppData\Local\Temp\ipykernel_20264\3458109425.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_delisted['폐지연도'] = filtered_delisted['DelistingDate'].dt.year
C:\Users\jeons\AppData\Local\Temp\ipykernel_20264\3458109425.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_delisted['폐지월'] = filtered_delisted['DelistingDate'].dt.month
C:\Users\jeons\AppData\Local\Temp\ipykernel_20264\3458109425.py:49: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice 